In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
import re

df = pd.read_csv('/content/Hotel_Reviews.csv')

In [ ]:
df['Average_Score'].unique()
df["Average_Score_Round"] = df["Average_Score"].apply(lambda x: int(round(x)))


In [ ]:
def label_reviews(row):
  review = row['Average_Score_Round']
  if(review == 6 and review == 5):
    return 1
  if(review == 7):
    return 2
  if(review == 8):
     return 3
  if(review == 9):
    return 4
  else:
      return 5


def createLabelsFromReviewPoints(df):
  df['Target']= df.apply (lambda row: label_reviews(row), axis=1)
  return df
  
df = createLabelsFromReviewPoints(df)


In [ ]:
df1 = df.sample(n = 160000) 

In [ ]:
df1["Review"] = df1["Negative_Review"] + df1["Positive_Review"]
#df1['Target'] = np.where(df1.eval("Average_Score_Round > 8"), "Good", "Bad")

In [ ]:
df_final = df1[['Review','Target','Negative_Review','Positive_Review']]# remove 'No Negative' or 'No Positive' from text
df_final["Negative_Review"] = df_final["Negative_Review"].apply(lambda x: x.replace("No Negative", ""))
df_final["Positive_Review"] = df_final["Positive_Review"].apply(lambda x: x.replace("No Positive", ""))

In [ ]:
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

In [ ]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#df["review_clean"] = df["Review"].apply(lambda x: clean_text(x))
df_final['Review'] = df_final['Review'].apply(lambda text: clean_text(text))

In [ ]:
#Splitting the data to train and test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df_final["Review"]
y = df_final["Target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords 
ENGLISH_STOP_WORDS = stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect_1 = TfidfVectorizer(min_df = 1, stop_words={'english'}, max_features = 8000,
                          ngram_range=(1,3)).fit(X_train)
X_train1 = vect_1.transform(X_train)
X_test1 = vect_1.transform(X_test)

In [ ]:
new_df_words = pd.DataFrame(columns=vect_1.get_feature_names(), data=X_train1.toarray())
new_df_words

In [ ]:
#counting most repetitive words 
word_counts = np.array(np.sum(X_train1, axis=0)).reshape((-1,))
words = np.array(vect_1.get_feature_names())
words_df = pd.DataFrame({"word":words, "count":word_counts})
words_df.sort_values(by="count",ascending=False).tail(50)


In [ ]:
# Visualizing in the form of Word Cloud
from wordcloud import WordCloud

text_pos = " ".join(i for i in words_df.word)
wc_pos = WordCloud(background_color="black", max_words=1000, contour_width=14, contour_color='firebrick')

# Generate a wordcloud
wc_pos.generate(text_pos)

plt.figure(figsize=(20,10))
plt.imshow(wc_pos, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# fitting a logistic regression model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
from sklearn import ensemble, linear_model, neighbors, svm, tree
from sklearn import svm, model_selection, tree, linear_model, neighbors, naive_bayes, ensemble
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Fitting Logistic regression to the training set
from sklearn.linear_model import LogisticRegressionCV
logreg1 = LogisticRegressionCV()
logreg1.fit(X_train1, y_train)

# Predicting the test set results
y_pred_logreg = logreg1.predict(X_test1)

pred_prob = logreg1.predict_proba(X_test1
                                  

# Confusion matrix and classifacation report for Logistic Regression 
from sklearn.metrics import classification_report
print('The Confusion Matrix')
con_mat_lr = confusion_matrix(y_test, y_pred_logreg)
df_cm_lr = pd.DataFrame(con_mat_lr, index = ['Star2','Star3','Star4',"Star5"], 
                   columns = ['Star2','Star3','Star4',"Star5"])
#display(con_mat_lr)
plt.figure(figsize=(10,6))
conf_mat_normalized = df_cm_lr.astype('float') / df_cm_lr.sum(axis=1)[:, np.newaxis]
sns.heatmap(df_cm_lr, annot=True,fmt='.1f')
plt.title("Confusion Matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show(block = False)


print('The Classification report')
report = classification_report(y_test, y_pred_logreg, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report



In [ ]:
# Loading Support Vector Classifier
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(X_train1, y_train)

# Predicting the test set results
y_pred_svc = svc.predict(X_test1)

# Loading COnfusion matrix and classifaction report 
print('The Confusion Matrix')
con_mat_svc = confusion_matrix(y_test, y_pred_svc)
#df_cm_svc = pd.DataFrame(con_mat_svc, columns = ['Predicted 0','Predicted 1'], index = ['True 0','True 1'])
display(con_mat_svc)

from sklearn.metrics import classification_report
print('The Confusion Matrix')
con_mat_svc = confusion_matrix(y_test, y_pred_svc)
df_cm_svc = pd.DataFrame(con_mat_svc, index = ['Star2','Star3','Star4',"Star5"], 
                   columns = ['Star2','Star3','Star4',"Star5"])
#display(con_mat_lr)
plt.figure(figsize=(10,6))
sns.heatmap(df_cm_svc, annot=True,fmt='.1f')
plt.title("Confusion Matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show(block = False)


print('The Classification report')
report = classification_report(y_test, y_pred_svc, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report

In [ ]:
# Building Multinomial Naive Bayes modle and fit it to our training set
from sklearn.naive_bayes import MultinomialNB
classifier1 = MultinomialNB(alpha=0.1)
classifier1.fit(X_train1, y_train)
pred = classifier1.predict(X_test)


# Confusion matrix and classifacation report for Multinomial Navie Bayes
from sklearn.metrics import classification_report
print('The Confusion Matrix')
con_mat_mb = confusion_matrix(y_test, pred)
#df_cm_lr = pd.DataFrame(con_mat_lr, columns = ['Predicted 0','Predicted 1'], index = ['True 0','True 1'])
display(con_mat_mb)

df_cm_mb = pd.DataFrame(con_mat_mb, index = ['Star2','Star3','Star4',"Star5"], 
                   columns = ['Star2','Star3','Star4',"Star5"])
plt.figure(figsize=(10,6))
sns.heatmap(df_cm_mb, annot=True,fmt='.1f')
plt.title("Confusion Matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show(block = False)

print('The Classification report')
report = classification_report(y_test, pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report

In [ ]:
# Loading Random Forest classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10,random_state=1)
rf.fit(X_train1, y_train)

# Predicting the test set results
y_pred_rf = rf.predict(X_test1)



# Confusion matrix and classifacation report for Random Forest classifier
from sklearn.metrics import classification_report
print('The Confusion Matrix')
con_mat_rf = confusion_matrix(y_test, y_pred_rf)
#df_cm_lr = pd.DataFrame(con_mat_lr, columns = ['Predicted 2','Predicted 3' ], index = ['True 0','True 1'])
display(con_mat_rf)

df_cm_lr = pd.DataFrame(con_mat_rf, index = ['Star2','Star3','Star4',"Star5"], 
                   columns = ['Star2','Star3','Star4',"Star5"])
plt.figure(figsize=(10,6))
sns.heatmap(df_cm_lr, annot=True,fmt='.1f')
plt.title("Confusion Matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show(block = False)

print('The Classification report')
report = classification_report(y_test, y_pred_rf, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_repor

In [ ]:
# Performance comparsion using Machine learning algorithms 
MLA = [
       linear_model.LogisticRegressionCV(),
       naive_bayes.MultinomialNB(),
       svm.LinearSVC(),
       ensemble.RandomForestClassifier(),  
]

In [ ]:
# Training the data into model and calculating performance 
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)

row_index = 0
for alg in MLA:
  predicted = alg.fit(X_train1, y_train).predict(X_test1)
  MLA_name = alg.__class__.__name__
  MLA_compare.loc[row_index,'MLA Name'] = MLA_name
  MLA_compare.loc[row_index, 'MLA Train Accuracy'] = round(alg.score(X_train1, y_train), 4)
  MLA_compare.loc[row_index, 'MLA Test Accuracy'] = round(alg.score(X_test1, y_test), 4)
  MLA_compare.loc[row_index, 'MLA Precission'] = precision_score(y_test, predicted,average='weighted')
  MLA_compare.loc[row_index, 'MLA Recall'] = recall_score(y_test, predicted,average='weighted')
  row_index+=1
    
MLA_compare.sort_values(by = ['MLA Test Accuracy'], ascending = False, inplace = True)    
MLA_compare


In [ ]:
plt.subplots(figsize=(12,6))
sns.barplot(x="MLA Name", y="MLA Train Accuracy",data=MLA_compare,palette='hot',edgecolor=sns.color_palette('dark',7))
plt.xticks(rotation=50)
plt.title('MLA Train Accuracy Comparison')
plt.show()

In [ ]:
plt.subplots(figsize=(12,6))
sns.barplot(x="MLA Name", y="MLA Test Accuracy",data=MLA_compare,palette='hot',edgecolor=sns.color_palette('dark',7))
plt.xticks(rotation=50)
plt.title('MLA Test Accuracy Comparison')
plt.show()

In [ ]:
prediction = pd.DataFrame(y_pred_logreg, columns=['Pred_Rating']).to_csv('prediction.csv')
df3 = pd.read_csv('/content/prediction (1).csv')

In [ ]:
labels = ["Star3","Star2","Star4"]
df3.Pred_Rating.value_counts().plot.pie(autopct='%1.1f%%',figsize=(8,10));
plt.legend(title="Ratings")